In [2]:
# Running the Colab Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np

In [3]:
# Opening the Dataframe "Bureau_Balance"

bb = pd.read_csv("C:\\Users\\kbc\Downloads\\home-credit-default-risk\\bureau_balance.csv")
bb.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


Analysing the data in mainlines

In [4]:
bb.describe().T

,count,mean,std,min,25%,50%,75%,max
SK_ID_BUREAU,27299925.0,6.036297e+06,492348.856904,5001709.0,5730933.0,6070821.0,6431951.0,6842888.0
MONTHS_BALANCE,27299925.0,-3.074169e+01,23.864509,-96.0,-46.0,-25.0,-11.0,0.0


Nan values

In [5]:
bb.isnull().sum()

SK_ID_BUREAU      0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [6]:
bb['STATUS'].unique(),bb['STATUS'].value_counts()

(array(['C', '0', 'X', '1', '2', '3', '5', '4'], dtype=object),
 C    13646993
 0     7499507
 X     5810482
 1      242347
 5       62406
 2       23419
 3        8924
 4        5847
 Name: STATUS, dtype: int64)

In [7]:
bb['MONTHS_BALANCE'].unique()

array([  0,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10, -11, -12,
       -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25,
       -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38,
       -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51,
       -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64,
       -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77,
       -78, -79, -80, -81, -82, -83, -84, -85, -86, -87, -88, -89, -90,
       -91, -92, -93, -94, -95, -96], dtype=int64)

In [8]:
def summary(df, pred=None):
    obs = df.shape[0]
    Types = df.dtypes
    Counts = df.apply(lambda x: x.count())
    Min = df.min()
    Max = df.max()
    Uniques = df.apply(lambda x: x.unique().shape[0])
    Nulls = df.apply(lambda x: x.isnull().sum())
    print('Data shape:', df.shape)

    if pred is None:
        cols = ['Types', 'Counts', 'Uniques', 'Nulls', 'Min', 'Max']
        str = pd.concat([Types, Counts, Uniques, Nulls, Min, Max], axis = 1, sort=True)

    str.columns = cols
    print('___________________________\nData Types:')
    print(str.Types.value_counts())
    print('___________________________')
    display(str.sort_values(by='Nulls', ascending=False))

In [9]:
summary(bb)

Data shape: (27299925, 3)
___________________________
Data Types:
int64     2
object    1
Name: Types, dtype: int64
___________________________


,Types,Counts,Uniques,Nulls,Min,Max
MONTHS_BALANCE,int64,27299925,97,0,-96,0
SK_ID_BUREAU,int64,27299925,817395,0,5001709,6842888
STATUS,object,27299925,8,0,0,X


PART1 : New FEATURE 1 counts_months by SK_ID_BUREAU

Groupby the client id (SK_ID_BUREAU), count of the number of STATUS, and rename the column

In [10]:
counts_months = bb.groupby('SK_ID_BUREAU', as_index=False)['MONTHS_BALANCE'].count().rename(columns = {'MONTHS_BALANCE': 'counts_months'})

In [11]:
counts_months.sample()

,SK_ID_BUREAU,counts_months
730109,6653837,3


In [12]:
def count_categorical(df, group_var, df_name):
    """Computes counts and normalized counts for each observation
    of `group_var` of each unique category in every categorical variable
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    group_var : string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with counts and normalized counts of each unique category in every categorical variable
        with one row for every unique value of the `group_var`.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

PART 2 for the database (16 features)

In [13]:
bb1 = count_categorical(bb, 'SK_ID_BUREAU', 'BB')

In [14]:
bb1.head()

,BB_STATUS_0_count,BB_STATUS_0_count_norm,BB_STATUS_1_count,BB_STATUS_1_count_norm,BB_STATUS_2_count,BB_STATUS_2_count_norm,BB_STATUS_3_count,BB_STATUS_3_count_norm,BB_STATUS_4_count,BB_STATUS_4_count_norm,BB_STATUS_5_count,BB_STATUS_5_count_norm,BB_STATUS_C_count,BB_STATUS_C_count_norm,BB_STATUS_X_count,BB_STATUS_X_count_norm
SK_ID_BUREAU,,,,,,,,,,,,,,,,
5001709,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402
5001710,5,0.060241,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446
5001711,3,0.750000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000
5001712,10,0.526316,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000
5001713,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000


In [15]:
summary(bb1)

Data shape: (817395, 16)
___________________________
Data Types:
uint8      8
float64    8
Name: Types, dtype: int64
___________________________


,Types,Counts,Uniques,Nulls,Min,Max
BB_STATUS_0_count,uint8,817395,98,0,0.0,97.000000
BB_STATUS_0_count_norm,float64,817395,2863,0,0.0,1.000000
BB_STATUS_1_count,uint8,817395,52,0,0.0,56.000000
BB_STATUS_1_count_norm,float64,817395,1187,0,0.0,1.000000
BB_STATUS_2_count,uint8,817395,19,0,0.0,39.000000
BB_STATUS_2_count_norm,float64,817395,420,0,0.0,0.714286
BB_STATUS_3_count,uint8,817395,12,0,0.0,18.000000
BB_STATUS_3_count_norm,float64,817395,249,0,0.0,0.400000
BB_STATUS_4_count,uint8,817395,13,0,0.0,12.000000
BB_STATUS_4_count_norm,float64,817395,201,0,0.0,0.500000


MERGING THE TWO PARTS

In [16]:
result = pd.merge(bb1, counts_months, on="SK_ID_BUREAU")

In [17]:
result.head()

,SK_ID_BUREAU,BB_STATUS_0_count,BB_STATUS_0_count_norm,BB_STATUS_1_count,BB_STATUS_1_count_norm,BB_STATUS_2_count,BB_STATUS_2_count_norm,BB_STATUS_3_count,BB_STATUS_3_count_norm,BB_STATUS_4_count,BB_STATUS_4_count_norm,BB_STATUS_5_count,BB_STATUS_5_count_norm,BB_STATUS_C_count,BB_STATUS_C_count_norm,BB_STATUS_X_count,BB_STATUS_X_count_norm,counts_months
0,5001709,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402,97
1,5001710,5,0.060241,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446,83
2,5001711,3,0.750000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000,4
3,5001712,10,0.526316,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000,19
4,5001713,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000,22


In [ ]:
summary(result)

Data shape: (817395, 18)
___________________________
Data Types:
float64    8
uint8      8
int64      2
Name: Types, dtype: int64
___________________________


,Types,Counts,Uniques,Nulls,Min,Max
BB_STATUS_0_count,uint8,817395,98,0,0.0,9.700000e+01
BB_STATUS_0_count_norm,float64,817395,2863,0,0.0,1.000000e+00
SK_ID_BUREAU,int64,817395,817395,0,5001709.0,6.842888e+06
BB_STATUS_X_count_norm,float64,817395,2866,0,0.0,1.000000e+00
BB_STATUS_X_count,uint8,817395,98,0,0.0,9.700000e+01
BB_STATUS_C_count_norm,float64,817395,2852,0,0.0,1.000000e+00
BB_STATUS_C_count,uint8,817395,98,0,0.0,9.700000e+01
BB_STATUS_5_count_norm,float64,817395,1281,0,0.0,1.000000e+00
BB_STATUS_5_count,uint8,817395,90,0,0.0,9.700000e+01
BB_STATUS_4_count_norm,float64,817395,201,0,0.0,5.000000e-01


a new feature 

In [18]:
result['status_rate'] = result['BB_STATUS_5_count_norm'] + result['BB_STATUS_4_count_norm'] + result['BB_STATUS_3_count_norm'] + result['BB_STATUS_2_count_norm'] + result['BB_STATUS_1_count_norm']

In [22]:
# function for diminishing memory used. (optional) 
import sys

# def return_size(df):
#     """Return size of dataframe in gigabytes"""
#     return round(sys.getsizeof(df) / 1e9, 2)

def convert_types(df, print_info = False):
    
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
        print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')
        
    return df

In [23]:
bb_ready = convert_types(result, print_info = False)

In [27]:
result.to_csv('/content/drive/MyDrive/Home Credit/home-credit-default-risk/bb_ready.csv', index=False)